Decision Tree
- 의사결정 나무
- 트리 구조
- 각 분기점(node)에는 분석대상(열)의 속성(설명변수) 위치

- 평가 지표 
    - Entropy 활용
        - 다른 종류의 값들이 섞여 있는 정도
        - 낮을수록 분류가 잘 된것
        - 일정 수준 이하로 낮아질때까지 반복

In [13]:
# <예제 7-6> Decision Tree 모형

import pandas as pd
import numpy as np

'''
step 1 데이터 준비 / 기본 설정
'''

# Breast Cancer 데이터셋 가져오기
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
breast-cancer-wisconsin/breast-cancer-wisconsin.data'

df = pd.read_csv(uci_path,header = None)
df.columns = ["id","clump","cell_size","cell_shape","adhesion","epithlial",
             "bare_nuclei","chromatin","normal_nucleoli","mitoses","class"]

# Ipython 디스플레이 설정 변경 - 출력할 열의 한도 개수 늘리기
pd.set_option("display.max_columns", 15)

'''
step 2 데이터 탐색
'''
# 데이터 살펴보기
print(df.head())
print("\n")
# 데이터 자료형 확인
print(df.info()) # bare_nuclei 열 처리 필요
# 데이터 통계 요약 정보 확인
print(df.describe())

# "bare_nuclei" 열의 자료형 변경(문자열 -> 정수형)
print(df["bare_nuclei"].unique()) # ? 처리 필요
print("\n")

df["bare_nuclei"].replace("?",np.nan,inplace = True)
df.dropna(subset = ["bare_nuclei"],axis = 0, inplace = True) # 누락 데이터 행 삭제
df["bare_nuclei"] = df["bare_nuclei"].astype("int") # 자료형 변경

print(df.describe())
print("\n")

'''
step 3 데이터 셋 구분 - 훈련용(train data) / 검증용(test data)
'''

#속성(변수)(열) 선택
# X(설명 변수)
X = df[["clump","cell_size","cell_shape","adhesion","epithlial",
       "bare_nuclei","chromatin","normal_nucleoli","mitoses"]]
# y(예측 변수)
y = df["class"]

# 설명 변수(X) 정규화
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data 를 7:3 으로 구분
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.3,
                                                   random_state = 10)

print("train data 개수:",X_train.shape)
print("test data 개수:",X_test.shape)

'''
step 4 Decision Tree 분류 모형 - sklearn 사용
- 1) sklearn 라이브러리의 tree 모듈 import 
- 2) DecisionTreeClassifier() 함수로 모형 객체 생성
    - 옵션 : 
        criterion = 분류 정도를 평가하는 기중
        max_depth = n 트리 레벨 지정
'''

from sklearn import tree

# 모형 객체 생성
tree_model = tree.DecisionTreeClassifier(criterion = "entropy",max_depth = 5)

# train data 로 모형 학습
tree_model.fit(X_train, y_train)

# test data 로 y_hat 예측(분류)
y_hat = tree_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics
tree_metrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_metrix)
print("\n")

# 모형 성능 평가 - 평가 지표 계산
tree_report = metrics.classification_report(y_test,y_hat)
print(tree_report)

        id  clump  cell_size  cell_shape  adhesion  epithlial bare_nuclei  \
0  1000025      5          1           1         1          2           1   
1  1002945      5          4           4         5          7          10   
2  1015425      3          1           1         1          2           2   
3  1016277      6          8           8         1          3           4   
4  1017023      4          1           1         3          2           1   

   chromatin  normal_nucleoli  mitoses  class  
0          3                1        1      2  
1          3                2        1      2  
2          3                1        1      2  
3          3                7        1      2  
4          3                1        1      2  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               699 non-null    int64 
 1   cl